In [ ]:
import os
import shutil

from zipfile import ZipFile

from spswarehouse.warehouse import Warehouse
from spswarehouse.googledrive import GoogleDrive

# Setup

It's easiest to move the two files from NSC into the same directory as this notebook.

In [ ]:
excel_file_path = '10055266hsst_10055266-152604-DETAIL-EFFDT-20220421-RUNDT-20220514.csv'
zip_file_path = '10055266hsst_10055266_EFFDT_20220421_RUNDT_20220514184645761.zip'

alumni_folder_id = '1Y3bSyRHwceFmsNYbo9vd59UXEtZHpj99'

upload_date = '2022_05'

In [ ]:
def UploadFileToDrive(filename, parent_folder_id):
    upload_file = GoogleDrive.CreateFile({
        'title': filename,
        'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}],
    })
    upload_file.SetContentFile(filename)
    upload_file.Upload()
    return upload_file

## Extract all files from the zip folder

In [ ]:
temp_dir = 'data'

nsc_zip = ZipFile(zip_file_path)

# Note: you can skip os.makedir because extractall creates the given path if it doesn't exist
nsc_zip.extractall(temp_dir)

nsc_zip.close()

## Retrieve list of CEEB codes

In [ ]:
ceeb_sql = """
SELECT DISTINCT
    site_short_name
    , ceeb_code
FROM public.sites_historical
"""

sites_df = Warehouse.read_sql(ceeb_sql)

## Create Drive folder for this upload, upload raw data files there

In [ ]:
folder_name = upload_date + '_nsc_files'

newFolder = GoogleDrive.CreateFile({
    'title': folder_name,
    "parents": [{"kind": "drive#fileLink", "id": alumni_folder_id}],
    "mimeType": "application/vnd.google-apps.folder"
})

newFolder.Upload()

In [ ]:
UploadFileToDrive(excel_file_path, newFolder['id'])
UploadFileToDrive(zip_file_path, newFolder['id'])

# Rename Files

In [ ]:
os.chdir(temp_dir)

In [ ]:
zip_file_list = os.listdir()
for file_name in zip_file_list:
    if 'ACADEMICS' in file_name:
        os.remove(file_name)
        continue

    if file_name[22:24] == 'HS':
        ceeb_code = file_name[24:30]
        site_name = sites_df[sites_df['ceeb_code']==ceeb_code]['site_short_name'].iloc[0]
        os.rename(file_name, site_name + file_name[37:])
    else:
        os.rename(file_name, 'Network' + file_name[37:])

# Upload renamed files to drive

In [ ]:
folder_name = upload_date + '_nsc_files'

newFolder = GoogleDrive.CreateFile({
    'title': folder_name,
    "parents": [{"kind": "drive#fileLink", "id": alumni_folder_id}],
    "mimeType": "application/vnd.google-apps.folder"
})

newFolder.Upload()

In [ ]:
# upload re-named files
upload_file_list = os.listdir()
for file_name in upload_file_list:
    file_upload = UploadFileToDrive(file_name, newFolder['id'])
    

# Cleanup

In [ ]:
# This might not work - I can't seem to close the connection to the last file uploaded

shutil.rmtree(temp_dir)